In [1]:
import sqlite3
import pandas as pd
import os
import argparse
import asyncio
from tqdm.notebook import tqdm
from db import DB

In [2]:
db = DB("dart_corp.db")
db.open_connection()

In [3]:
folder_path = "../data"
year = 2023
quarter = 1
exctv_file_name = f"2021_2022-상장사-drop_dup-{2023}년도-{1}분기.csv"
corp_file_name = f"2021_2022-상장사-drop_dup.csv"
exctv_file_path = os.path.join(folder_path, exctv_file_name)
corp_file_path = os.path.join(folder_path, corp_file_name)

drop_cols = ['발행주식총수', '시가총액', '종가', '연도']
exctv_df = pd.read_csv(exctv_file_path, encoding='utf-8', dtype=object).drop(columns=drop_cols)
exctv_df['기간'] = str(year) + "_" + str(quarter) 
corp_df = pd.read_csv(corp_file_path, encoding='utf-8', dtype=object, index_col=0).drop(columns=['연도'])

In [4]:
exctv_df

,종목코드,종목명,시장구분,corp_code,이름,생년월일,성별,직위,등기여부,상근여부,담당업무,최대주주와의관계,재직기간,임기만료일,기간
0,005930,삼성전자,KOSPI,00126380,한종희,1962년 03월,남,부회장,사내이사,상근,ㆍ대표이사 (DX 부문 경영전반 총괄),계열회사임원,37개월,2026년 03월 15일,2023_1
1,005930,삼성전자,KOSPI,00126380,허은녕,1964년 08월,남,이사,사외이사,비상근,ㆍ사외이사후보추천위원회 위원ㆍ지속가능경영위원회 위원,계열회사임원,5개월,2025년 11월 02일,2023_1
2,005930,삼성전자,KOSPI,00126380,유명희,1967년 06월,여,이사,사외이사,비상근,ㆍ사외이사후보추천위원회 위원ㆍ지속가능경영위원회 위원,계열회사임원,5개월,2025년 11월 02일,2023_1
3,005930,삼성전자,KOSPI,00126380,경계현,1963년 03월,남,사장,사내이사,상근,ㆍ대표이사 (DS 부문 경영전반 총괄),계열회사임원,13개월,2025년 03월 15일,2023_1
4,005930,삼성전자,KOSPI,00126380,노태문,1968년 09월,남,사장,사내이사,상근,ㆍMX사업부장,계열회사임원,13개월,2025년 03월 15일,2023_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29733,114570,지스마트글로벌,KOSDAQ,00673976,박승만,1946년 08월,남,감사,감사,비상근,감사,없음,1년2개월,2023년 10월 29일,2023_1
29734,240340,인터코스,KONEX,01117592,김주덕,1970년 11월,남,대표이사,사내이사,상근,경영총괄,대표이사,-,2024년 03월 29일,2023_1
29735,240340,인터코스,KONEX,01117592,김경렬,1970년 04월,남,관리이사,사내이사,상근,총무,-,-,2025년 03월 31일,2023_1
29736,240340,인터코스,KONEX,01117592,김봉수,1970년 01월,남,영업이사,사내이사,상근,영업,특수관계자,-,2024년 03월 29일,2023_1


In [5]:
corp_df

,종목코드,종목명,시장구분,corp_code
0,005930,삼성전자,KOSPI,00126380
1,373220,LG에너지솔루션,KOSPI,01515323
2,207940,삼성바이오로직스,KOSPI,00877059
3,000660,SK하이닉스,KOSPI,00164779
4,051910,LG화학,KOSPI,00356361
...,...,...,...,...
2615,240340,인터코스,KONEX,01117592
2616,329020,오션스톤,KONEX,01388631
2617,225850,미애부,KONEX,00866594
2618,221610,자안바이오,KOSDAQ,00493431


## DB TABLE 생성

In [6]:
create_table_query_corp = '''CREATE TABLE corp (
                            corp_code TEXT PRIMARY KEY,
                            stock_code TEXT,
                            stock_name TEXT,
                            market TEXT )
                        '''

create_table_query_exc_list = '''CREATE TABLE executive_list (
                        corp_code TEXT,
                        pid TEXT,
                        year INTEGER,
                        quarter INTEGER,
                        position TEXT,
                        registration TEXT,
                        full_time TEXT,
                        job TEXT,
                        relationship_with_shareholder TEXT,
                        employ_duration TEXT,
                        expire_date TEXT,
                        PRIMARY KEY (corp_code, pid, year, quarter),
                        CONSTRAINT corp_code_fk FOREIGN KEY(corp_code) REFERENCES corp(corp_code),
                        CONSTRAINT pid_fk FOREIGN KEY(pid) REFERENCES executive_details(pid)
                        );
                        '''
create_table_query_exc_det = '''CREATE TABLE executive_details (
                        pid TEXT PRIMARY KEY,
                        name TEXT,
                        birth_date TEXT,
                        sex TEXT);
                        '''

In [7]:
db.run_query(create_table_query_corp)
db.run_query(create_table_query_exc_det)
db.run_query(create_table_query_exc_list)

OperationalError: table corp already exists

In [8]:
print(db.run_fetch("SELECT name FROM sqlite_master WHERE type='table';"))

[('corp',), ('executive_details',), ('executive_list',)]


## INSERT datas

In [9]:
insert_query_corp = '''INSERT INTO corp
                    (corp_code, stock_code, stock_name, market) 
                    VALUES (?, ?, ?, ?)'''

insert_query_exc_det = '''INSERT INTO executive_details
                        (pid, name, birth_date, sex) 
                        VALUES (?, ?, ?, ?)'''

insert_query_exc_list = '''INSERT INTO executive_list
                        (corp_code, pid, year, quarter, position, 
                        registration, full_time, job, relationship_with_shareholder, 
                        employ_duration, expire_date) 
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''


#### INSERT TO corp

In [10]:
corp_df.to_records(index=False)

rec.array([('005930', '삼성전자', 'KOSPI', '00126380'),
           ('373220', 'LG에너지솔루션', 'KOSPI', '01515323'),
           ('207940', '삼성바이오로직스', 'KOSPI', '00877059'), ...,
           ('225850', '미애부', 'KONEX', '00866594'),
           ('221610', '자안바이오', 'KOSDAQ', '00493431'),
           ('225860', '엠앤씨생명과학', 'KONEX', '01116274')],
          dtype=[('종목코드', 'O'), ('종목명', 'O'), ('시장구분', 'O'), ('corp_code', 'O')])

In [11]:
db.run_query("DELETE FROM corp")
corp_df = corp_df[['corp_code', '종목코드', '종목명', '시장구분']]
for row in corp_df.to_records(index=False):
    
    db.run_query_with_params(insert_query_corp, row)

#### INSERT TO exc_det (map pid)

In [12]:
def map_pid(df: pd.DataFrame, db: DB):
    db.run_query("DELETE FROM executive_details")
    idtfy_df = df[:][['이름', '생년월일', '성별']]
    idtfy_df = idtfy_df.drop_duplicates()
    
    query = "SELECT PID FROM executive_details WHERE (name = ? and birth_date = ? and sex = ?)"
    for pid, row in idtfy_df.iterrows():
        
        params = list(row.values)
        res = db.run_fetch_with_params(query, params)
        if res is not None:
            params = [str(pid).zfill(6)] + [x for x in params]
            params[2] = params[2].replace(" ", "")
            db.run_query_with_params(insert_query_exc_det, params) 

map_pid(exctv_df, db)

In [13]:
print(db.run_fetch("SELECT * FROM executive_details"))

[('000000', '한종희', '1962년03월', '남'), ('000001', '허은녕', '1964년08월', '남'), ('000002', '유명희', '1967년06월', '여'), ('000003', '경계현', '1963년03월', '남'), ('000004', '노태문', '1968년09월', '남'), ('000005', '박학규', '1964년11월', '남'), ('000006', '이정배', '1967년02월', '남'), ('000007', '김한조', '1956년07월', '남'), ('000008', '김선욱', '1952년12월', '여'), ('000009', '김종훈', '1960년08월', '남'), ('000010', '김준성', '1967년10월', '남'), ('000011', '권영수', '1957년02월', '남'), ('000012', '이방수', '1958년11월', '남'), ('000013', '장성훈', '1979년01월', '남'), ('000014', '장승권', '1969년07월', '남'), ('000015', '장응진', '1968년11월', '남'), ('000016', '정왕모', '1971년10월', '남'), ('000017', '정원희', '1975년09월', '남'), ('000018', '정하상', '1971년01월', '남'), ('000019', '정혁성', '1973년07월', '남'), ('000020', '정회국', '1977년08월', '남'), ('000021', '조상연', '1977년02월', '남'), ('000022', '최대식', '1975년03월', '남'), ('000023', '김수령', '1962년03월', '남'), ('000024', '최상훈', '1972년02월', '남'), ('000025', '최성훈', '1968년12월', '남'), ('000026', '최세호', '1966년06월', '남'), ('000027', '최유라', '1976년12월

#### INSERT TO exctv_list 

###### update exctv_df pid and will insert to executive_list_df

In [14]:
query = f"SELECT pid FROM executive_details WHERE name = ? and birth_date = ? and sex = ?"

exctv_det_df = exctv_df[:][['이름','생년월일','성별']]
exctv_det_df['생년월일'] = exctv_det_df['생년월일'].apply(lambda x: x.replace(" ", ""))

for pid, row in exctv_det_df.iterrows():
    params = row[['이름', '생년월일', '성별']].values
    res = db.run_fetch_with_params(query, params)
    if len(res) > 0:
        exctv_df.loc[pid, 'pid'] = str(res[0][0]).zfill(6)    

In [15]:
exctv_df.head()

,종목코드,종목명,시장구분,corp_code,이름,생년월일,성별,직위,등기여부,상근여부,담당업무,최대주주와의관계,재직기간,임기만료일,기간,pid
0,005930,삼성전자,KOSPI,00126380,한종희,1962년 03월,남,부회장,사내이사,상근,ㆍ대표이사 (DX 부문 경영전반 총괄),계열회사임원,37개월,2026년 03월 15일,2023_1,000000
1,005930,삼성전자,KOSPI,00126380,허은녕,1964년 08월,남,이사,사외이사,비상근,ㆍ사외이사후보추천위원회 위원ㆍ지속가능경영위원회 위원,계열회사임원,5개월,2025년 11월 02일,2023_1,000001
2,005930,삼성전자,KOSPI,00126380,유명희,1967년 06월,여,이사,사외이사,비상근,ㆍ사외이사후보추천위원회 위원ㆍ지속가능경영위원회 위원,계열회사임원,5개월,2025년 11월 02일,2023_1,000002
3,005930,삼성전자,KOSPI,00126380,경계현,1963년 03월,남,사장,사내이사,상근,ㆍ대표이사 (DS 부문 경영전반 총괄),계열회사임원,13개월,2025년 03월 15일,2023_1,000003
4,005930,삼성전자,KOSPI,00126380,노태문,1968년 09월,남,사장,사내이사,상근,ㆍMX사업부장,계열회사임원,13개월,2025년 03월 15일,2023_1,000004


In [23]:
exctv_list_df = exctv_df[:]
exctv_list_df = exctv_list_df.drop(columns=['이름', '생년월일', '성별'])

db.run_query("DELETE FROM executive_list")

for i, row in exctv_list_df.iterrows():
    params = row[['corp_code', 'pid']].values.tolist()
    params2 = row['기간'].split('_')
    params3 = row[['직위', '등기여부', '상근여부', '담당업무', '최대주주와의관계', '재직기간', '임기만료일']].values.tolist()
    params.extend(params2)
    params.extend(params3)
    try:
        db.run_query_with_params(insert_query_exc_list, params)
    except Exception as e:
        print(db.run_fetch_with_params("SELECT * FROM executive_list WHERE pid = ?", [params[1]]))
        print(db.run_fetch_with_params("SELECT * FROM executive_details WHERE pid = ?", [params[1]]))
        print(db.run_fetch_with_params("SELECT * FROM corp WHERE corp_code = ?", [params[0]]))
        print(params, end="\n------------------------------------------------------------------------\n")

[('00154462', '003702', 2023, 1, '경영자', '미등기', '상근', '회장실', '-', '2015.01.01~', '-')]
[('003702', '이준식', '1970년02월', '남')]
[('00154462', '002790', '아모레G', 'KOSPI')]
['00154462', '003702', '2023', '1', '경영자', '미등기', '상근', '비서실', '-', '2015.01.01~', '-']
------------------------------------------------------------------------
[('00154462', '003703', 2023, 1, '경영자', '미등기', '상근', '전략기획', '계열회사 임원', '2014.09.01 ~', '-')]
[('003703', '이진표', '1977년11월', '남')]
[('00154462', '002790', '아모레G', 'KOSPI')]
['00154462', '003703', '2023', '1', '경영자', '미등기', '상근', '전략기획', '계열회사 임원', '2014.09.01 ~', '-']
------------------------------------------------------------------------
[('00105855', '005066', 2023, 1, '이사', '미등기', '상근', '글로벌인사실장(CHO)', '-', '1993.01.01~', '-')]
[('005066', '최종섭', '1968년02월', '남')]
[('00105855', '010120', 'LS ELECTRIC', 'KOSPI')]
['00105855', '005066', '2023', '1', '이사', '미등기', '상근', '글로벌인사실장(CHO)', '-', '1993.01.01~', '-']
--------------------------------------------------------

In [17]:
print(db.run_fetch_with_params("SELECT * FROM executive_list WHERE pid = ?", ["021809"]))
print(db.run_fetch_with_params("SELECT * FROM executive_details WHERE pid = ?", ["021809"]))
print(db.run_fetch_with_params("SELECT * FROM corp WHERE corp_code = ?", ["00616290"]))

[('00616290', '021809', 2023, 1, '이사', '사외이사', '비상근', '사외이사감사위원', '등기임원', '2021.04.01~2023.03.29', '-')]
[('021809', '신승훈', '1975년03월', '남')]
[('00616290', '123570', '이엠넷', 'KOSDAQ')]


In [24]:
print(db.run_fetch("SELECT * FROM corp"))

[('00126380', '005930', '삼성전자', 'KOSPI'), ('01515323', '373220', 'LG에너지솔루션', 'KOSPI'), ('00877059', '207940', '삼성바이오로직스', 'KOSPI'), ('00164779', '000660', 'SK하이닉스', 'KOSPI'), ('00356361', '051910', 'LG화학', 'KOSPI'), ('00126362', '006400', '삼성SDI', 'KOSPI'), ('00164742', '005380', '현대차', 'KOSPI'), ('00266961', '035420', 'NAVER', 'KOSPI'), ('00106641', '000270', '기아', 'KOSPI'), ('00258801', '035720', '카카오', 'KOSPI'), ('00155319', '005490', 'POSCO홀딩스', 'KOSPI'), ('00413046', '068270', '셀트리온', 'KOSPI'), ('00149655', '028260', '삼성물산', 'KOSPI'), ('00688996', '105560', 'KB금융', 'KOSPI'), ('00164788', '012330', '현대모비스', 'KOSPI'), ('00382199', '055550', '신한지주', 'KOSPI'), ('00631518', '096770', 'SK이노베이션', 'KOSPI'), ('00126256', '032830', '삼성생명', 'KOSPI'), ('00401731', '066570', 'LG전자', 'KOSPI'), ('00181712', '034730', 'SK', 'KOSPI'), ('00159193', '015760', '한국전력', 'KOSPI'), ('00155276', '003670', '포스코케미칼', 'KOSPI'), ('00244455', '033780', 'KT&G', 'KOSPI'), ('00547583', '086790', '하나금융지주', 'KOSPI'

In [17]:
print(db.run_fetch("SELECT * FROM executive_details"))

[('000000', '한종희', '1962년03월', '남'), ('000001', '허은녕', '1964년08월', '남'), ('000002', '유명희', '1967년06월', '여'), ('000003', '경계현', '1963년03월', '남'), ('000004', '노태문', '1968년09월', '남'), ('000005', '박학규', '1964년11월', '남'), ('000006', '이정배', '1967년02월', '남'), ('000007', '김한조', '1956년07월', '남'), ('000008', '김선욱', '1952년12월', '여'), ('000009', '김종훈', '1960년08월', '남'), ('000010', '김준성', '1967년10월', '남'), ('000011', '권영수', '1957년02월', '남'), ('000012', '이방수', '1958년11월', '남'), ('000013', '장성훈', '1979년01월', '남'), ('000014', '장승권', '1969년07월', '남'), ('000015', '장응진', '1968년11월', '남'), ('000016', '정왕모', '1971년10월', '남'), ('000017', '정원희', '1975년09월', '남'), ('000018', '정하상', '1971년01월', '남'), ('000019', '정혁성', '1973년07월', '남'), ('000020', '정회국', '1977년08월', '남'), ('000021', '조상연', '1977년02월', '남'), ('000022', '최대식', '1975년03월', '남'), ('000023', '김수령', '1962년03월', '남'), ('000024', '최상훈', '1972년02월', '남'), ('000025', '최성훈', '1968년12월', '남'), ('000026', '최세호', '1966년06월', '남'), ('000027', '최유라', '1976년12월

In [25]:
db.commit_connection()
db.close_connection()

## Add Datas

In [ ]:
db = DB("dart_corp.db")
db.open_connection()

folder_path = "../data"
year = 2023
quarter = 1
exctv_file_name = f"2021_2022-상장사-drop_dup-{2023}년도-{1}분기.csv"
corp_file_name = f"2021_2022-상장사-drop_dup.csv"
exctv_file_path = os.path.join(folder_path, exctv_file_name)
corp_file_path = os.path.join(folder_path, corp_file_name)

drop_cols = ['발행주식총수', '시가총액', '종가', '연도']
exctv_df = pd.read_csv(exctv_file_path, encoding='utf-8', dtype=object).drop(columns=drop_cols)
exctv_df['기간'] = str(year) + "_" + str(quarter) 
corp_df = pd.read_csv(corp_file_path, encoding='utf-8', dtype=object, index_col=0).drop(columns=['연도'])

In [ ]:
insert_query_exc_list = '''INSERT INTO executive_list
                        (corp_code, pid, year, quarter, position, 
                        registration, full_time, job, relationship_with_shareholder, 
                        employ_duration, expire_date) 
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''